# Creating Quarterly Financial Statement DataFrames

Welcome! This notebook shows you how to create normalized pandas DataFrames containing quarterly financial statements (Income Statement, Balance Sheet, and Cash Flow) for multiple companies over the last 5 years.

This tutorial was inspired by a community question about extracting structured financial data from SEC filings. We'll show you how **edgartools** makes this surprisingly simple.

## What We'll Build

By the end of this notebook, you'll have:
- Three DataFrames (Income Statement, Balance Sheet, Cash Flow)
- Multi-index structure: `(Ticker, Period)`
- Columns for all line items from the financial statements
- Quarterly data (not annual) for the last 5 years

## The Key API: Facts

The **Facts API** (`company.get_facts()`) is your gateway to structured financial data. It returns quarterly statements directly as pandas DataFrames, with proper period handling that elegantly handles the "10-K is full year" problem.

## Setup

First, let's install edgartools if you haven't already:

In [ ]:
!pip install -U edgartools

Now import the necessary libraries and set your identity. The SEC requires you to identify yourself when accessing their API:

In [ ]:
from edgar import set_identity, Company
import pandas as pd

# Replace with your actual name and email
set_identity("Your Name your@email.com")

## Step 1: Single Company — Quick Look

Let's start by getting financial statements for one company. This shows you what the data looks like before we build the multi-company DataFrames.

In [ ]:
# Get Apple's company data
company = Company("AAPL")
facts = company.get_facts()

# Display the last 4 quarters of income statement
# This gives you a nicely formatted view of the data
income = facts.income_statement(periods=4, annual=False)
print(income)

Notice how clean and readable this is! The `annual=False` parameter ensures we get **quarterly** data (Q1, Q2, Q3, Q4) instead of annual totals from 10-K filings.

## Step 2: Single Company — As DataFrame

Now let's get the same data as a pandas DataFrame. We'll request 20 periods (5 years × 4 quarters):

In [ ]:
# Get 5 years of quarterly income statement as a DataFrame
income_df = facts.income_statement(periods=20, annual=False, as_dataframe=True)
income_df.head()

### Understanding the DataFrame Structure

The DataFrame has:
- **Index**: XBRL concept names (e.g., "Revenues", "CostOfRevenue")
- **Metadata columns**: `label`, `depth`, `is_abstract`, `is_total`, `section`, `confidence`
- **Period columns**: "Q1 2024", "Q2 2024", etc. — these contain the actual values

Let's inspect the shape:

In [ ]:
print(f"Shape: {income_df.shape}")
print(f"\nPeriod columns (showing first 5):")
period_cols = [c for c in income_df.columns if c not in ['label', 'depth', 'is_abstract', 'is_total', 'section', 'confidence']]
print(period_cols[:5])

## Step 3: Multiple Tickers — Building the Multi-Index DataFrame

Now for the main event! Let's create the multi-index DataFrame structure you requested:
- **Index**: `(Ticker, Period)`
- **Columns**: Financial statement line items

We'll start with the Income Statement for three companies:

In [ ]:
tickers = ["AAPL", "MSFT", "GOOGL"]

# Build income statement DataFrame with multi-index
income_frames = []

for ticker in tickers:
    print(f"Fetching data for {ticker}...")
    company = Company(ticker)
    facts = company.get_facts()
    
    # Get quarterly income statement
    df = facts.income_statement(periods=20, annual=False, as_dataframe=True)
    
    # Keep only the period columns (drop metadata)
    period_cols = [c for c in df.columns if c not in ['label', 'depth', 'is_abstract', 'is_total', 'section', 'confidence']]
    
    # Transpose: periods become rows, line items become columns
    df_t = df[period_cols].T
    df_t.columns = df['label']  # Use readable labels as column names
    df_t.index.name = 'Period'
    
    # Add ticker column and set multi-index
    df_t['Ticker'] = ticker
    df_t = df_t.reset_index().set_index(['Ticker', 'Period'])
    
    income_frames.append(df_t)

# Combine all companies
income_all = pd.concat(income_frames)
print(f"\nFinal DataFrame shape: {income_all.shape}")
income_all.head(10)

Perfect! Now you have a multi-index DataFrame with all quarterly income statements. Let's do the same for Balance Sheet and Cash Flow:

In [ ]:
# Balance Sheet
balance_frames = []

for ticker in tickers:
    print(f"Fetching balance sheet for {ticker}...")
    company = Company(ticker)
    facts = company.get_facts()
    df = facts.balance_sheet(periods=20, annual=False, as_dataframe=True)
    period_cols = [c for c in df.columns if c not in ['label', 'depth', 'is_abstract', 'is_total', 'section', 'confidence']]
    df_t = df[period_cols].T
    df_t.columns = df['label']
    df_t.index.name = 'Period'
    df_t['Ticker'] = ticker
    df_t = df_t.reset_index().set_index(['Ticker', 'Period'])
    balance_frames.append(df_t)

balance_all = pd.concat(balance_frames)
print(f"\nBalance Sheet shape: {balance_all.shape}")
balance_all.head(10)

In [ ]:
# Cash Flow Statement
cashflow_frames = []

for ticker in tickers:
    print(f"Fetching cash flow for {ticker}...")
    company = Company(ticker)
    facts = company.get_facts()
    df = facts.cashflow_statement(periods=20, annual=False, as_dataframe=True)
    period_cols = [c for c in df.columns if c not in ['label', 'depth', 'is_abstract', 'is_total', 'section', 'confidence']]
    df_t = df[period_cols].T
    df_t.columns = df['label']
    df_t.index.name = 'Period'
    df_t['Ticker'] = ticker
    df_t = df_t.reset_index().set_index(['Ticker', 'Period'])
    cashflow_frames.append(df_t)

cashflow_all = pd.concat(cashflow_frames)
print(f"\nCash Flow shape: {cashflow_all.shape}")
cashflow_all.head(10)

## Step 4: Wrap It Into a Reusable Function

Now let's create a clean, reusable function that does all this work for you:

In [ ]:
def get_quarterly_statements(tickers, periods=20):
    """
    Get quarterly financial statements as DataFrames with multi-index (Ticker, Period).
    
    Parameters
    ----------
    tickers : list of str
        Company ticker symbols (e.g., ['AAPL', 'MSFT', 'GOOGL'])
    periods : int, default=20
        Number of quarterly periods to retrieve (20 = 5 years)
    
    Returns
    -------
    dict
        Dictionary with keys 'income_statement', 'balance_sheet', 'cashflow'
        Each value is a DataFrame with multi-index (Ticker, Period)
    """
    results = {}
    
    for statement_name, method_name in [
        ('income_statement', 'income_statement'),
        ('balance_sheet', 'balance_sheet'),
        ('cashflow', 'cashflow_statement'),
    ]:
        print(f"\nProcessing {statement_name}...")
        frames = []
        
        for ticker in tickers:
            print(f"  {ticker}...", end=" ")
            company = Company(ticker)
            facts = company.get_facts()
            method = getattr(facts, method_name)
            df = method(periods=periods, annual=False, as_dataframe=True)
            
            # Extract period columns only
            period_cols = [c for c in df.columns if c not in ['label', 'depth', 'is_abstract', 'is_total', 'section', 'confidence']]
            
            # Transpose and format
            df_t = df[period_cols].T
            df_t.columns = df['label']
            df_t.index.name = 'Period'
            df_t['Ticker'] = ticker
            df_t = df_t.reset_index().set_index(['Ticker', 'Period'])
            frames.append(df_t)
            print("Done")
        
        results[statement_name] = pd.concat(frames)
        print(f"  Final shape: {results[statement_name].shape}")
    
    return results

Now let's test it with a different set of companies from various industries:

In [ ]:
# Tech, Retail, Healthcare
companies = ["NVDA", "WMT", "JNJ"]

statements = get_quarterly_statements(companies, periods=20)

# Access individual statements
income_df = statements['income_statement']
balance_df = statements['balance_sheet']
cashflow_df = statements['cashflow']

In [ ]:
# Example: Get revenue for all companies across all periods
if 'Revenues' in income_df.columns:
    income_df['Revenues'].dropna()

In [ ]:
# Example: Get data for a specific company and period
income_df.loc[('NVDA', 'Q1 2024')]

## Tips & Next Steps

### Other Parameters You Can Use

- **`annual=True`**: Get annual data instead of quarterly
- **`periods=40`**: Get more historical data (10 years of quarterly data)
- **`concise_format=True`**: Use when displaying statements (removes extra detail)

### Working with the Data

```python
# Filter to specific ticker
aapl_income = income_all.loc['AAPL']

# Filter to specific period
q1_2024 = income_all.xs('Q1 2024', level='Period')

# Get a specific line item across all companies/periods
revenues = income_all['Revenues'].dropna()
```

### Export to CSV

```python
income_all.to_csv('quarterly_income_statements.csv')
balance_all.to_csv('quarterly_balance_sheets.csv')
cashflow_all.to_csv('quarterly_cashflow_statements.csv')
```

### Learn More

- [edgartools Documentation](https://edgartools.github.io/edgartools/)
- [Facts API Guide](https://edgartools.github.io/edgartools/concepts/facts/)
- [Financial Statements Guide](https://edgartools.github.io/edgartools/guides/working-with-financials/)

---

**You did it!** You now have a clean, reusable workflow for extracting quarterly financial statements into normalized pandas DataFrames. This pattern works for any publicly traded company in the SEC database.